In [2]:
import numpy as np
import random
import math

# -------------------------------
# Problem Setup
# -------------------------------
def target_function(x):
    return np.exp(x)

def taylor_series(x, coeffs):
    return sum(coeffs[i] * (x ** i) for i in range(len(coeffs)))

def mse_error(coeffs, xs):
    ys_true = target_function(xs)
    ys_pred = taylor_series(xs, coeffs)
    return np.mean((ys_true - ys_pred) ** 2)

# -------------------------------
# PCA Functions
# -------------------------------
def random_solution(n_terms, bounds):
    return np.array([random.uniform(bounds[0], bounds[1]) for _ in range(n_terms)])

def mutate(coeffs, bounds, mutation_rate=0.1):
    new = coeffs.copy()
    for i in range(len(coeffs)):
        if random.random() < mutation_rate:
            new[i] += random.uniform(-0.05, 0.05)
            new[i] = np.clip(new[i], bounds[0], bounds[1])
    return new

def get_neighbors(grid, i, j):
    nrows, ncols = grid.shape[0], grid.shape[1]
    neighbors = []
    if i > 0: neighbors.append(grid[i-1][j])
    if i < nrows-1: neighbors.append(grid[i+1][j])
    if j > 0: neighbors.append(grid[i][j-1])
    if j < ncols-1: neighbors.append(grid[i][j+1])
    return neighbors

# -------------------------------
# Parallel Cellular Algorithm
# -------------------------------
def parallel_cellular_algorithm_2D(nrows=7, ncols=7, n_terms=4, bounds=(-2,2),
                                   Tmax=200, lam=0.3, p_mut=0.1):
    xs = np.linspace(-1, 1, 200)

    # Step 1: Initialize population
    grid = np.empty((nrows, ncols), dtype=object)
    fitness = np.zeros((nrows, ncols))
    for i in range(nrows):
        for j in range(ncols):
            grid[i][j] = random_solution(n_terms+1, bounds)
            fitness[i][j] = -mse_error(grid[i][j], xs)

    # Step 2: Iterative Optimization
    for t in range(Tmax):
        new_grid = np.empty_like(grid)
        for i in range(nrows):
            for j in range(ncols):
                neighbors = get_neighbors(grid, i, j)
                best_neighbor = max(neighbors, key=lambda c: -mse_error(c, xs))
                current = grid[i][j]

                # Diffusion update (Von Neumann neighborhood)
                if -mse_error(best_neighbor, xs) > -mse_error(current, xs):
                    diff = np.zeros_like(current)
                    for n in neighbors:
                        diff += (n - current)
                    new = current + lam * diff / len(neighbors)
                else:
                    new = current

                # Mutation
                if random.random() < p_mut:
                    new = mutate(new, bounds)

                new_grid[i][j] = new
        grid = new_grid

    # Step 3: Find best solution
    all_cells = [grid[i][j] for i in range(nrows) for j in range(ncols)]
    best = min(all_cells, key=lambda c: mse_error(c, xs))
    best_mse = mse_error(best, xs)

    return best, best_mse

# -------------------------------
# Run the algorithm
# -------------------------------
best_coeffs, best_error = parallel_cellular_algorithm_2D()

print("✅ Optimal Taylor Coefficients Found:")
for i, a in enumerate(best_coeffs):
    print(f"a{i} = {a:.6f}")

print(f"\nMean Squared Error: {best_error:.10f}")

# Compare true vs optimized coefficients (for e^x centered at 0)
true_coeffs = [1/math.factorial(i) for i in range(len(best_coeffs))]
print("\n📘 True Taylor Coefficients (e^x):")
for i, a in enumerate(true_coeffs):
    print(f"a{i} = {a:.6f}")


✅ Optimal Taylor Coefficients Found:
a0 = 0.879473
a1 = 0.532670
a2 = 0.274348
a3 = 0.395308
a4 = 0.789272

Mean Squared Error: 0.0596915817

📘 True Taylor Coefficients (e^x):
a0 = 1.000000
a1 = 1.000000
a2 = 0.500000
a3 = 0.166667
a4 = 0.041667
